In [1]:
''# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/galaxy10/images.npy
/kaggle/input/galaxy10/labels.npy


***Hello everyone, in this notebook I am going to use semi-supervised learning approach for the image analysis of the Galaxy10 dataset. It is a use dataset of shape 21785 * 69 * 69. Here
I am using VAE along with a custom ANN model,We basically trained VAE on a minimum reconstruction loss, then extract the encoder part ,freeze its weights and on the base of the encoder we use a dense layer for the task of the image classification.***

# Loading of the datasets

In [2]:
train1=np.load("/kaggle/input/galaxy10/images.npy")

In [3]:
target=np.load('/kaggle/input/galaxy10/labels.npy')

In [4]:
target.shape

(21785,)

In [4]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow as tf
import keras
from keras import layers

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Sampling Layer

In [5]:

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


# Architecture of the Encoder

In [48]:
from tensorflow.keras import layers

latent_dim = 10

# Define your custom layers
inputs = keras.Input(shape=(69, 69, 3))  # Fix the input layer
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="sigmoid")(x)

# Continue with your code
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = layers.Lambda(Sampling(), output_shape=(latent_dim,), name="sampling")([z_mean, z_log_var])  # Assuming you have a Sampling layer or a Lambda layer named 'sampling'

# Build the model
encoder = keras.Model(inputs=inputs, outputs=[z_mean, z_log_var, z], name="encoder")  # Adjust the base_model input
encoder.summary()


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 69, 69, 3)]          0         []                            
                                                                                                  
 conv2d_21 (Conv2D)          (None, 35, 35, 64)           1792      ['input_15[0][0]']            
                                                                                                  
 batch_normalization_43 (Ba  (None, 35, 35, 64)           256       ['conv2d_21[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 conv2d_22 (Conv2D)          (None, 18, 18, 64)           36928     ['batch_normalization_43

# Architecture of the decoder

In [49]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(9*9*32, activation="relu")(latent_inputs)
x = layers.Reshape((9, 9, 32))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x=layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
x=layers.BatchNormalization()(x)
x = layers.Conv2DTranspose(32, 3, activation="sigmoid", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder_outputs = layers.Cropping2D(cropping=((3, 0), (3, 0)), data_format=None)(x)  # Adjust cropping

# Crop or pad to get the desired shape 69*69*3

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 10)]              0         
                                                                 
 dense_37 (Dense)            (None, 2592)              28512     
                                                                 
 reshape_8 (Reshape)         (None, 9, 9, 32)          0         
                                                                 
 conv2d_transpose_35 (Conv2  (None, 18, 18, 64)        18496     
 DTranspose)                                                     
                                                                 
 batch_normalization_45 (Ba  (None, 18, 18, 64)        256       
 tchNormalization)                                               
                                                                 
 conv2d_transpose_36 (Conv2  (None, 36, 36, 32)        1846

# VAE Model

In [32]:

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


# Train-Test Split

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train1, target, test_size=0.2, random_state=42)

In [34]:
df = np.concatenate([X_train, X_test], axis=0)
df = df.astype("float32") / 255

# Training to the minimum reconstruction loss

In [50]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
history=vae.fit(mnist_digits, epochs=10, batch_size=32)

Epoch 1/10
681/681 [==============================] - 15s 13ms/step - loss: 1337.0352 - reconstruction_loss: 1215.1970 - kl_loss: 6.6338
Epoch 2/10
681/681 [==============================] - 9s 13ms/step - loss: 1172.9487 - reconstruction_loss: 1160.6001 - kl_loss: 7.8416
Epoch 3/10
681/681 [==============================] - 9s 13ms/step - loss: 1163.1930 - reconstruction_loss: 1157.9260 - kl_loss: 7.4413
Epoch 4/10
681/681 [==============================] - 9s 13ms/step - loss: 1161.0558 - reconstruction_loss: 1155.7097 - kl_loss: 7.6172
Epoch 5/10
681/681 [==============================] - 8s 12ms/step - loss: 1161.3546 - reconstruction_loss: 1153.8108 - kl_loss: 7.8816
Epoch 6/10
681/681 [==============================] - 9s 13ms/step - loss: 1159.4403 - reconstruction_loss: 1152.9047 - kl_loss: 8.0152
Epoch 7/10
681/681 [==============================] - 8s 12ms/step - loss: 1162.7206 - reconstruction_loss: 1152.3563 - kl_loss: 8.0935
Epoch 8/10
681/681 [===========================

In [51]:
encoder.trainable=False# freeze the weights

In [46]:
encoder.output

[<KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'z_mean')>,
 <KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'z_log_var')>,
 <KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'sampling')>]

# Dense layer for the image classification

In [56]:
# Assuming you have the pretrained 'encoder' model
encoded_vectors = encoder.get_layer('flatten_6').output  # Assuming the layer is named 'z'

# Add a fully connected layer on top of the encoded vectors
classification_layer = layers.Dense(256, activation='relu')(encoded_vectors)
classification_layer=layers.BatchNormalization()(classification_layer)
layer1=layers.Dense(128, kernel_regularizer=keras.regularizers.l2(0.001),activation='relu')(classification_layer)
layer1=layers.Dropout(0.5)(layer1)
layer2=layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001),activation='relu')(layer1)
layer2=layers.Dropout(0.5)(layer2)
layer3=layers.Dense(32, kernel_regularizer=keras.regularizers.l2(0.001),activation='relu')(layer2)
layer3=layers.BatchNormalization()(layer3)
# Add a separate fully connected layer for the sampling output (optional)

# Add the final output layer for classification
output_layer = layers.Dense(10, activation='softmax')(layer3)

# Create a new model with the frozen encoder input and the output_layer as output
classification_model = keras.Model(inputs=encoder.input, outputs=output_layer, name='classification_model')

# Compile the model with an appropriate optimizer and loss for the MNIST classification task
classification_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
def lr_scheduler(epoch,lr):
    if(epoch<=50):
        lr=0.001
    elif(epoch>50 & epoch <75):
        lr=0.0001
    else:
        lr=0.00001
    return lr
# Create a LearningRateScheduler callback
lr_scheduler_callback = keras.callbacks.LearningRateScheduler(lr_scheduler)
# Train the new model
history=classification_model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test),callbacks=[lr_scheduler_callback])

# Evaluate the model on the test set
accuracy = classification_model.evaluate(X_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/100
545/545 [==============================] - 6s 7ms/step - loss: 1.9735 - accuracy: 0.4198 - val_loss: 1.4972 - val_accuracy: 0.5433 - lr: 0.0010
Epoch 2/100
545/545 [==============================] - 3s 6ms/step - loss: 1.4660 - accuracy: 0.5483 - val_loss: 1.2991 - val_accuracy: 0.5703 - lr: 0.0010
Epoch 3/100
545/545 [==============================] - 3s 6ms/step - loss: 1.3241 - accuracy: 0.5957 - val_loss: 1.3983 - val_accuracy: 0.5419 - lr: 0.0010
Epoch 4/100
545/545 [==============================] - 3s 6ms/step - loss: 1.2354 - accuracy: 0.6140 - val_loss: 1.2620 - val_accuracy: 0.5977 - lr: 0.0010
Epoch 5/100
545/545 [==============================] - 3s 6ms/step - loss: 1.1681 - accuracy: 0.6294 - val_loss: 1.1083 - val_accuracy: 0.6438 - lr: 0.0010
Epoch 6/100
545/545 [==============================] - 3s 6ms/step - loss: 1.1055 - accuracy: 0.6395 - val_loss: 1.0596 - val_accuracy: 0.6502 - lr: 0.0010
Epoch 7/100
545/545 [==============================] - 3s 6ms/st

In [57]:
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=history.epoch, y=history.history['accuracy'], mode='lines', name='Train accuracy'))
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['val_accuracy'], mode='lines', name='Validation accuracy'))

fig.update_layout(title='Training and Validation accuracy Over Epochs',
                  xaxis=dict(title='Epochs'),
                  yaxis=dict(title='accuracy'),
                  template='plotly_dark')

# Show the plot
fig.show()


# Results

We can see there is little bit overfitting but when I used simple CNN ,even I have used Transfer Learning I am just able to get 72% accuracy with 66% validation accuracy,but using these semi-supervised learning we get better results ,which is an amazing thing.